In [1]:
import cv2  
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16

In [2]:
# Main directory where model data is saved
mainDataDirectory = "C:/Users/seyed/Music/Brain Tumor - Model Data/"

In [3]:
# Categories of image classification
imageCategories = ["Yes", "No"]

# Directories for training and testing data
trainDirectory = os.path.join(mainDataDirectory, 'Train')
testDirectory = os.path.join(mainDataDirectory, 'Test')

In [4]:
# Function to read images from a directory and add labels to each image
def read_images_from_directory(directory):
    images = []
    labels = []
    for category in imageCategories:
        category_path = os.path.join(directory, category)
        label = imageCategories.index(category)
        
        for filename in os.listdir(category_path):
            image_path = os.path.join(category_path, filename)
            image = cv2.imread(image_path)
            images.append(image)
            if label == 0:
                labels.append("Yes")
            elif label == 1:
                labels.append("No")
    return images, labels

In [5]:
# Read training images
train_x, train_y = read_images_from_directory(trainDirectory)

# Read testing images
test_x, test_y = read_images_from_directory(testDirectory)

# Convert the lists to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

In [6]:
# Checking for the number of data
train_x.shape

(41104, 256, 256, 3)

In [7]:
test_x.shape

(10276, 256, 256, 3)

In [9]:
# Checking the categories in the dataset
np.unique(train_y)

array(['No', 'Yes'], dtype='<U3')

In [10]:
# Map string labels to integer labels inorder for the model to read the data in integer format
label_mapping = {'No': 1, 'Yes': 0}

# Convert string labels to integer labels
train_y = [label_mapping[label] for label in train_y]
test_y = [label_mapping[label] for label in test_y]

# Convert labels to one-hot encoded format
train_y_one_hot = to_categorical(train_y, num_classes=2)
test_y_one_hot = to_categorical(test_y, num_classes=2)

In [11]:
# Running the tf in cpu
with tf.device('/cpu:0'):
    # Loading the vgg16 model and setting up the image size to train
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    
    # Freeze the layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Build the model
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
    # Train the model
    history = model.fit(train_x, train_y_one_hot, epochs=10, batch_size=16, validation_data=(test_x, test_y_one_hot))


Epoch 1/10
2569/2569 [==============================] - 2268s 883ms/step - loss: 0.4637 - accuracy: 0.9429 - val_loss: 0.0392 - val_accuracy: 0.9877
Epoch 2/10
2569/2569 [==============================] - 2265s 882ms/step - loss: 0.1314 - accuracy: 0.9553 - val_loss: 0.0428 - val_accuracy: 0.9880
Epoch 3/10
2569/2569 [==============================] - 2265s 882ms/step - loss: 0.1192 - accuracy: 0.9622 - val_loss: 0.0246 - val_accuracy: 0.9919
Epoch 4/10
2569/2569 [==============================] - 2260s 880ms/step - loss: 0.1124 - accuracy: 0.9665 - val_loss: 0.0470 - val_accuracy: 0.9887
Epoch 5/10
2569/2569 [==============================] - 2267s 882ms/step - loss: 0.0959 - accuracy: 0.9712 - val_loss: 0.0796 - val_accuracy: 0.9848
Epoch 6/10
2569/2569 [==============================] - 2271s 884ms/step - loss: 0.0880 - accuracy: 0.9736 - val_loss: 0.0187 - val_accuracy: 0.9946
Epoch 7/10
2569/2569 [==============================] - 2261s 880ms/step - loss: 0.1003 - accuracy: 0.9710

In [12]:
# Save the model
model.save('C:/Users/seyed/Music/Brain Tumor - Model/TumorDetectionModel_VGG-16.h5')

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 256)               8388864   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 23,104,066
Trainable params: 8,389,378
Non-trainable params: 14,714,688
_________________________________________________________________
